In [ ]:
import paramiko
from watchdog.observers import Observer
from watchdog.events import PatternMatchingEventHandler

import os
import errno
import configparser

In [ ]:
config = configparser.ConfigParser()
config.read(os.path.join('..', 'lib', 'utilities', 'ssh_config.ini'))

In [ ]:
rsa_key = paramiko.RSAKey.from_private_key_file(config['VIRTUALBOX']['pkey'])

ssh_client = paramiko.SSHClient()
ssh_client.set_missing_host_key_policy(paramiko.AutoAddPolicy())

print ('Connecting...')
ssh_client.connect(hostname=config['VIRTUALBOX']['host'], 
                   username=config['VIRTUALBOX']['user'], 
                   port=config['VIRTUALBOX']['port'],
                   pkey=rsa_key)
print ('Connected.')

In [ ]:
sftp_client = ssh_client.open_sftp()

In [ ]:
ssh_client.close()

In [ ]:
def mkdir_p(sftp, remote, is_dir=False):
    """
    emulates mkdir_p if required. 
    sftp - is a valid sftp object
    remote - remote path to create. 
    """
    dirs_ = []
    if is_dir:
        dir_ = remote
    else:
        dir_, _ = os.path.split(remote)
    while len(dir_) > 1:
        dirs_.append(dir_)
        dir_, _  = os.path.split(dir_)

    if len(dir_) == 1 and not dir_.startswith("/"): 
        dirs_.append(dir_) # For a remote path like y/x.txt 

    while len(dirs_):
        dir_ = dirs_.pop()
        try:
            sftp.stat(dir_)
        except:
            print (f'making ... dir {dir_}')
            sftp.mkdir(dir_)

In [ ]:
mkdir_p(sftp_client, './test1/test2', True)

In [ ]:
class ServerWorkSync(PatternMatchingEventHandler):

    def __init__(self, sftp_client, localpath, remotepath, patterns=None, ignore_patterns=None, ignore_directories=False, case_sensitive=False):   
        super(ServerWorkSync, self).__init__(patterns, ignore_patterns, ignore_directories, case_sensitive)    
        self.localpath = localpath
        self.root = os.path.split(localpath)[1]
        self.remotepath = remotepath
        self.sftp_client = sftp_client
        
        self.__handshake()

        
    def __directory_exists(self, path):
        'os.path.exists for paramiko SCP object'
        try:
            self.sftp_client.stat(path)
        except IOError as e:
            if e.errno == errno.ENOENT:
                return False
            raise
        else:
            return True

        
    def __cwd_scp(self, localpath, remotepath):
        #  recursively upload a full directory
        tmp = os.getcwd()
        os.chdir(os.path.split(localpath)[0])

        for walker in os.walk(self.root):
            try:
                self.sftp_client.mkdir(os.path.join(remotepath,walker[0]))
            except:
                pass
            for file in walker[2]:
                print (f'\tCopying {os.path.join(walker[0],file)}...')
                self.sftp_client.put(os.path.join(walker[0],file),os.path.join(remotepath,walker[0],file)) 
        os.chdir(tmp)
    
    
    def __handshake(self):
        direxists = self.__directory_exists(os.path.join(self.remotepath, os.path.split(self.localpath)[1]))
        
        if not direxists:
            print ("> Initiating Handshake. Transferring All Data to SSH Server...")
            self.__cwd_scp(self.localpath, self.remotepath)
        else:
            # TODO: Make handhake on existing directory.
            # Steps:
            #       1.   Compare Directories
            #       2.1. Copy new Files
            #       2.2. Replace old Files with newer ones
            pass   
        
        
    def on_moved(self, event):
        super(ServerWorkSync, self).on_moved(event)

        what = 'directory' if event.is_directory else 'file'
        print(f'Moved {what}: from {event.src_path} to {event.dest_path}')
        
        try:
            self.sftp_client.posix_rename(os.path.join(self.remotepath, self.root, ''.join(event.src_path.split(self.root, 1)[1:]).strip('/')), 
                                          os.path.join(self.remotepath, self.root, ''.join(event.dest_path.split(self.root, 1)[1:]).strip('/')))
        except FileNotFoundError:
            pass
    
    
    def on_created(self, event):
        super(ServerWorkSync, self).on_created(event)

        what = 'directory' if event.is_directory else 'file'
        print(f'Created {what}: {event.src_path}')
        
        try:
            if event.is_directory:
                self.sftp_client.mkdir(os.path.join(self.remotepath, self.root, ''.join(event.src_path.split(self.root, 1)[1:]).strip('/')))
            else:
                self.sftp_client.put(event.src_path,
                                     os.path.join(self.remotepath, self.root, ''.join(event.src_path.split(self.root, 1)[1:]).strip('/')),
                                     callback=None, confirm=True)
        except FileNotFoundError:
            pass

        
    def on_deleted(self, event):
        super(ServerWorkSync, self).on_deleted(event)

        what = 'directory' if event.is_directory else 'file'
        print(f'Deleted {what}: {event.src_path}')
        
        try:
            if event.is_directory:
                self.sftp_client.rmdir(os.path.join(self.remotepath, self.root, ''.join(event.src_path.split(self.root, 1)[1:]).strip('/')))  
            else:
                self.sftp_client.remove(os.path.join(self.remotepath, self.root, ''.join(event.src_path.split(self.root, 1)[1:]).strip('/')))  
        except FileNotFoundError:
            pass
        
        
    def on_modified(self, event):
        super(ServerWorkSync, self).on_modified(event)
        
        what = 'directory' if event.is_directory else 'file'
        print(f'Modified {what}: {event.src_path}')
        
        try:
            if event.is_directory:
                # NOTE: idk if this event is useful for directories, so i'll leave it for future use.
                pass
            else:
                self.sftp_client.put(event.src_path,
                                     os.path.join(self.remotepath, self.root, ''.join(event.src_path.split(self.root, 1)[1:]).strip('/')),
                                     callback=None, confirm=True)
        except FileNotFoundError:
            pass

* ## &nbsp;&nbsp;&nbsp;&nbsp; if __name__ == '__main__':

In [ ]:
import time

In [ ]:
#### GET THE HOME DIRECTORY OF THE SSH SERVER
stdin, stdout, stderr = ssh_client.exec_command("echo $HOME")

ssh_server_home_dir = stdout.readlines()[0].split('\n')[0]
ssh_server_home_dir

In [ ]:
ssh_client_localpath = os.path.abspath(".")
ssh_client_localpath

In [ ]:
handler = ServerWorkSync(sftp_client, localpath = ssh_client_localpath, remotepath = ssh_server_home_dir)  
observer = Observer()
observer.schedule(handler, path = ssh_client_localpath, recursive = True)
observer.start()

try:
    while True:
        time.sleep(1)
except KeyboardInterrupt:
    observer.stop()
finally:
    observer.join()

In [ ]:
os.stat(os.path.join('.', 'SSH_Connection.ipynb')).st_mtime

In [ ]:
def __remote_os_walk(root):
    import stat
    files = []
    dirs = []
    
    for f in sftp_client.listdir_attr(root):
        if stat.S_ISDIR(f.st_mode):
            dirs.append(f.filename)
        else:
            files.append(f.filename)
    yield root, dirs, files
    for folder in dirs:
        for x in __remote_os_walk(__unix_path(root, folder)):
            yield x

def __unix_path(*args):
    """Most handle UNIX pathing, not vice versa, enforce standard"""
    return os.path.join(*args).replace('\\', '/')

In [ ]:
import pprint
pp = pprint.PrettyPrinter(indent=4)

for root, dirs, files in __remote_os_walk('./RemoteCompProject'):
    print(f'\n{root}')
    print(f'{root.split("RemoteCompProject", 1)}\n')
    files = [os.path.join(root, _file) for _file in files]    
    for file in files:
        print (file, end='\t\t')
        print (datetime.utcfromtimestamp(int(sftp_client.stat(file).st_mtime)).strftime('%Y-%m-%d %H:%M:%S'))
        print (sftp_client.stat(file).st_mtime)

    

In [ ]:
for root, dirs, files in os.walk(os.path.abspath('.')):
    print(f'\n{root}')
#     print(f'{root.split("RemoteCompProject", 1)}\n')
    files = [os.path.join(root, _file) for _file in files]    
    for file in files:
        print (file, end='\t\t\n')
#         print (datetime.utcfromtimestamp(int(sftp_client.stat(file).st_mtime)).strftime('%Y-%m-%d %H:%M:%S'))
#         print (sftp_client.stat(file).st_mtime)

In [ ]:
from datetime import datetime

#### Here is how you can list all the directories.
for i in sftp_client.listdir():
    lstatout=str(sftp_client.lstat(i)).split()[0]
    if 'd' in lstatout: 
        print(datetime.utcfromtimestamp(int(sftp_client.stat(i).st_mtime)).strftime('%Y-%m-%d %H:%M:%S'), end='')
        print ('\t\t', i)

In [ ]:
#### Files are the opposite search:
for i in sftp_client.listdir():
    lstatout=str(sftp_client.lstat(i)).split()[0]
    if 'd' not in lstatout:
        print (f'{i}, is a file')